In [3]:
import pandas as pd
import numpy as np

In [4]:
data= pd.read_excel('dummy_data.xlsx')

In [6]:
data['Percolation(mm)'] = data['Percolation (m^3)']/ data['Area (m^2)'] *1000


In [51]:

#############################################################################
            #Initialize P
#############################################################################
"""
Organic phosphorus levels are assigned assuming that the N:P ratio for 
humic materials is 8:1
humic_orgN: Concentration of humic organic N in the layer (mg/kg) 
humic_orgP1: Concentration of humic organic P in the layer (mg/kg) 
"""
#humic_orgN = 300
humic_orgP1 = 0.125*humic_orgN

#fresh_orgP: P in the fresh organic pool in layer (kg P/ha)
fresh_orgP = 2 ##################

#fresh_orgP: N in the fresh organic pool in layer (kg N/ha)
fresh_orgN = 20 ##################

#NO3: amount of nitrate in layer (kg N/ha)
NO3 = 8 ##################

#solP: Amount of phosphorus in solution (mg/kg)
solP1 = 1


#layer bulk density (Mg/m3)
B_d = 1.3 #Bulk density values should fall between 1.1 and 1.9 Mg/m3.

#lyr_dpth: the depth of the layer (mm).
lyr_dpth = 200


# convert conc to mass (Kg P /ha)
solP = solP1 * B_d * lyr_dpth/100
humic_orgP = humic_orgP1* B_d * lyr_dpth/100

In [52]:
"""
humic_orgN: concentration  of  humic  organic  nitrogen  in  the  layer (mg/kg or ppm)
orgC: amount of organic carbon in the layer (%)
"""
orgC = 30 ###################
humic_orgN = 10**4 * (orgC/14)

In [53]:
"""
active_orgN: concentration  of  nitrogen  in  the  active  organic  pool (mg/kg)
stable_orgN: concentration  of  nitrogen  in  the  stable  organic  pool (mg/kg)
Fr: The fraction of humic nitrogen in the active pool
"""
Fr = 0.02 #default
active_orgN = humic_orgN * Fr
stable_orgN = humic_orgN * (1-Fr)

In [50]:
#############################################################################
            #MINERALIZATION & DECOMPOSITION/IMMOBILIZATION
#############################################################################

"""
Two  sources  are considered  for  mineralization:  the  fresh  organic  
P  pool  associated  with  crop residue and microbial biomass and the active
organic P pool associated with soil humus.  Mineralization  and  decomposition 
are  allowed  to  occur  only  if  the temperature of the soil layer
is above 0 degreeC. Mineralization and decomposition are dependent on water 
availability and temperature.  Two  factors  are  used  in  the  mineralization
and  decomposition equations to account for the impact of temperature and water
on these processes. 
NCTF: nutrient cycling temperature factor for each layer [not allowed to be smaller than 0.1]
NCWF: nutrient cycling water factor for each layer [not allowed to be smaller than 0.05]
soilT: temperature  of each layer in degreeC 
wc: soil water content for a given layer on a given day (mm)
fc: water content of a given layer at field capacity on a given day (mm)
"""

#NCTF = 0.9 * soilT/ soilT + np.exp(9.93-0.312*soilT) + 0.1
NCTF = 0.1 #########################
#NCWF = wc/fc
NCWF=0.05  #########################

In [56]:
"""
HUMUS MINERALIZATION
active_orgP:  amount of phosphorus in the active organic pool (kg P/ha)
stable_orgP:  amount of phosphorus in the stable organic pool (kg P/ha)
minP_humicorgP: the phosphorus mineralized from the humus active organic P pool (kg  P/ha)
B_min: rate  coefficient  for  mineralization  of  the  humus  active organic nutrients
Phosphorus mineralized from the humus active organic pool is added to the
solution P pool in the layer
"""

active_orgP = humic_orgP * (active_orgN/active_orgN + stable_orgN)
stable_orgP = humic_orgP * (stable_orgN/active_orgN + stable_orgN)

B_min = 0.5 ##############################

minP_humicorgP = 1.4* B_min * np.sqrt(NCTF*NCWF) * active_orgP

In [49]:
"""
Decomposition and mineralization 
Allowed only in first soil layer and controlled by a decay rate constant that 
is updated daily. 
The decay rate constant is  calculated  as  a  function  of  the  C:N  ratio 
and  C:P  ratio  of  the  residue, temperature and soil water content. 
E_cn: C:N ratio of the residue n the soil layer 
rsd: residue in layer ly (kg/ha)
0.58: fraction of residue that is carbon
fresh_orgN: nitrogen in the fresh organic pool in layer (kg N/ha)
NO3: amount of nitrate in layer (kg N/ha). 
E_cp: C:P ratio of the residue n the soil layer
solP : amount of phosphorus in solution in layer (kg P/ha)
fresh_orgP: hosphorus in the fresh organic pool in layer (kg P/ha).
"""
rsd=1 ###############################

E_cn = 0.58 * rsd /fresh_orgN + NO3

E_cp = 0.58 * rsd /fresh_orgP + solP


"""
DECAY RATE CONSTANT
The decay rate constant defines the fraction of residue that is decomposed. 
d_rate_const: decay rate constant 
B_rsd: The  fraction  of  residue  which  will  decompose  in  a  day assuming 
optimal  moisture,  temperature,  C:N  ratio  and  C:P ratio (default=0.05).
"""
a = np.exp(-0.693*(E_cn - 25)/25)
b = np.exp(-0.693*(E_cp - 200)/200)
c = [a,b,1]

NCRC = min(c)

B_rsd = 0.05

d_rate_const =  B_rsd * NCRC * np.sqrt(NCTF*NCWF)

#minP_freshorgP : Mineralization from the residue fresh organic P pool 
#decP_freshorgP : decomposition from the residue fresh organic P pool
minP_freshorgP = 0.8 * d_rate_const * fresh_orgP

decP_freshorgP = 0.2 * d_rate_const * fresh_orgP

In [20]:
#############################################################################
            #SORPTION OF INORGANIC P
#############################################################################

"""
SWAT assumes a rapid equilibrium exists between solution  P  and  an  “active”
mineral  pool. slow  reaction  is simulated  by  the  slow  equilibrium  
assumed  to  exist  between  the  “active”  and “stable”  mineral  pools. 
Equilibration between the solution and active mineral pool is governed by 
the phosphorus availability index (PAI)
solnP_f: amount  of phosphorus in solution after fertilization and incubation
solnP_i: amount  of phosphorus in solution before fertilization 
fert_minP: the amount of soluble P fertilizer added to the sample

PAI is calulated as:
PAI = solnP_f - solnP_i/fert_minP

if the value is not provided, default value is set to 0.4"""
PAI = 0.4

In [22]:
"""
MOVEMENT BETWEEN ACTIVE MINERAL POOL AND SOLUTION
The  movement  of  phosphorus  between  the  solution  and  active  mineral 
pools is governed by the equilibration equations: 
P_trans_sol_active_P: Amount of phosphorus transferred between the soluble and 
active mineral pool (kg P/ha). Positive value indicates transfer from 
solution to active mineral pool and vice a versa.
solP:  Amount of phosphorus in solution (kg P/ha)
active_minP: amount of phosphorus in the active mineral pool (kg P/ha)
"""
active_minP = 0.02 #####################################
minP=0.4 ####################################

if solP > minP * (PAI/(1-PAI)):
    P_trans_sol_active_P = 0.1*(solP - active_minP * (PAI/(1-PAI)))
    #print P_trans_sol_active_P
elif solP < minP * (PAI/(1-PAI)):
    P_trans_sol_active_P = 0.6*(solP - active_minP * (PAI/(1-PAI)))
    #print P_trans_sol_active_P

0.258666666667


In [27]:
"""
MOVEMENT BETWEEN STABLE MINERAL POOL AND SOLUTION
When not in equilibrium, the movement of phosphorus between the active 
and stable mineral pools is governed by the equations:
B_eqp:: slow equilibrium constant set to 0.0006 per day
"""
stable_minP = 0.2 ######################################
B_eqp= 0.0006 
if stable_minP < 4 * active_minP:
    P_trans_sol_stable_P = B_eqp * (4 * active_minP - stable_minP)
elif stable_minP > 4 * active_minP:
    P_trans_sol_stable_P = 0.1 * B_eqp * (4 * active_minP - stable_minP)

In [11]:
W_prec_surf = data['Percolation(mm)']

In [14]:
########## P LEACHING

#P_perc : Amount of phosphorus moving from the top 10 mm into the first 
#soil layer (kg P/ha)
#solP_10mm : amount of phosphorus in solution in the top 10 mm (kg P/ha)
#W_prec_surf:  amount of water percolating to the first soil layer 
#from the top 10 mm on a given day (mm)
#B_d: bulk density of the top 10mm  (Mg/m3) (assumed  to  be  equivalent  to  
#bulk  density  of  first  soil  layer)
#dpth: depth of the “surface” layer (10 mm)
#Kd_perc: phosphorus percolation  coefficient  (m3/Mg)

W_perc_surf = data['Percolation(mm)']
solP_10mm = 1
B_d = 1.3 #Bulk density values should fall between 1.1 and 1.9 Mg/m3. 
dpth = 200
Kd_perc = 10 #The value can range from 10.0 to 17.5. default is 10

P_perc = (solP_10mm * W_prec_surf) / (10 * B_d * dpth * Kd_perc)

In [16]:
data['P_percolation(Kg P /ha)'] = P_perc
